In [2]:
import re
import json
import csv
import time

from IPython.display import display
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from requests_html import HTMLSession

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from msedge.selenium_tools import Edge, EdgeOptions

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def scrape(ticker):
  "This opens the edge driver with on a given url"
  #Create the urls
  url_statistics = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
  url_financials = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
  url_historical = f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'


  driver.get(url_financials)
  time.sleep(3)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  pattern = re.compile(r'\s--\sData\s--\s')
  script_data = soup.find('script', text=pattern).contents[0]

  start = script_data.find("context")-2
  end = script_data.find("root.YAHOO ||")-2

  #data = [json.loads(line) for line in open('extra.json','r')]
  
  json_data = json.loads(script_data[start:end])

  json_data['context'].keys()

  annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
  quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

  annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
  quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

  annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
  quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

  qis = pd.json_normalize(quarterly_is)
  qis['ticker'] = ticker
  qis['endDate'] = qis['endDate.fmt']
  qcf = pd.json_normalize(quarterly_cf)
  qcf['ticker'] = ticker
  qbs = pd.json_normalize(quarterly_bs)
  qbs['ticker'] = ticker


  result = pd.merge(qis, qcf, on=["ticker", "endDate.raw"])

  end_result = pd.merge(result, qbs, on=["ticker", "endDate.raw"])

  return end_result

In [4]:
def scrape_stats(ticker):

    #Create the urls
    url_statistics = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    url_financials = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
    url_historical = f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'


    driver.get(url_statistics)
    time.sleep(3)

    # button = driver.find_element_by_xpath('//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span')
    # button.click()
    # time.sleep(0.5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]

    start = script_data.find("context")-2
    end = script_data.find("root.YAHOO ||")-2
    
    json_data = json.loads(script_data[start:end])

    keys = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'].keys()

    #loop through all the keys in the timeSeries store
    for key in keys:
        #the keys are broken into two categories, annual and trailing (twelve months)
        if (key[:9] =='quarterly'):
            #copy the json data for the key
            temp_json = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'][key]
            check_data = pd.json_normalize(temp_json)
            if (not check_data.empty):
                if (check_data['asOfDate'].isnull().values.any()):
                    print('skipped ' + key)
                else:
                    collected_data = check_data['asOfDate']
                    print('found a full date column')
            else:
                pass
        else:
            print('skipped ' + key)

    #loop through all the keys in the timeSeries store
    for key in keys:
        #the keys are broken into two categories, annual and trailing (twelve months)
        if (key[:9] =='quarterly'):
            #copy the json data for the key
            temp_json = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'][key]
            temp_data = pd.json_normalize(temp_json)
            #replace the generic value column name with the key name
            if (not temp_data.empty):
                temp_data = temp_data.rename(columns={'reportedValue.raw': key})
                temp_data = temp_data[key]
                print(f'about to merge ' + key + ' table')
                collected_data = pd.merge(collected_data, temp_data, left_index=True, right_index=True)
                print(f'succesfully added ' + key + ' table to the collection.')
            else:
                pass
            
        else:
            print('skipped ' + key)
    collected_data['ticker'] = ticker
    return (collected_data)



In [5]:
def scrape_income(ticker):

    #Create the urls
    url_statistics = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    url_financials = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
    url_historical = f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'


    driver.get(url_financials)
    time.sleep(3)

    #button = driver.find_elements_by_xpath("//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div")
    #'//div[@class="css-ais6tt"]//button[3]'
    #//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span

    # button = driver.find_element_by_xpath('//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span')
    # button.click()
    # time.sleep(0.5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]

    start = script_data.find("context")-2
    end = script_data.find("root.YAHOO ||")-2

    #data = [json.loads(line) for line in open('extra.json','r')]
    
    json_data = json.loads(script_data[start:end])

    keys = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'].keys()

    #loop through all the keys in the timeSeries store
    for key in keys:
        #the keys are broken into two categories, annual and trailing (twelve months)
        if (key[:6] =='annual'):
            #copy the json data for the key
            temp_json = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'][key]
            check_data = pd.json_normalize(temp_json)
            if (not check_data.empty):
                if (check_data['asOfDate'].isnull().values.any()):
                    print('skipped ' + key)
                else:
                    collected_data = check_data['asOfDate']
                    print('found a full date column')
            else:
                pass
        else:
            print('skipped ' + key)

    #loop through all the keys in the timeSeries store
    for key in keys:
        #the keys are broken into two categories, annual and trailing (twelve months)
        if (key[:6] =='annual'):
            #copy the json data for the key
            temp_json = json_data['context']['dispatcher']['stores']['QuoteTimeSeriesStore']['timeSeries'][key]
            temp_data = pd.json_normalize(temp_json)
            #replace the generic value column name with the key name
            if (not temp_data.empty):
                temp_data = temp_data.rename(columns={'reportedValue.raw': key})
                temp_data = temp_data[key]
                print(f'about to merge ' + key + ' table')
                collected_data = pd.merge(collected_data, temp_data, left_index=True, right_index=True)
                print(f'succesfully added ' + key + ' table to the collection.')
            else:
                pass
            
        else:
            print('skipped ' + key)
    collected_data['ticker'] = ticker
    return(collected_data)

In [6]:
def download_everything(ticker):
    #Create the urls
    url_statistics = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    url_balance = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
    url_flow = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'
    url_financials = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
    url_historical = f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'


    driver.get(url_financials)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/span/button').click()
    time.sleep(1)
    driver.get(url_balance)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/span/button').click()
    time.sleep(1)
    driver.get(url_flow)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/button/div/span').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-Financials-Proxy"]/section/div[1]/div[2]/span/button').click()
    time.sleep(1)
    driver.get(url_statistics)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div[1]/span/button').click()
    time.sleep(1)
    driver.get(url_historical)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/ul[2]/li[4]/button').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a').click()
    time.sleep(1)


In [7]:
def download_list(list_of_stocks):
    length = len(list_of_stocks)
    counter = 0
    counter2 = 0
    for stock in list_of_stocks:
        try:
            download_everything(stock)
        except:
            print(f"error downloading data for {stock}")
        counter += 1
        counter2 += 1
        print(f'{round((counter/length*100), 2)}% complete...')
        if (counter2 == 10):
            counter2 = 0
            time.sleep(140)
    print('All files downloaded!')

In [8]:
def remove_fluff(df):
    for col_name in df.columns:
        if (col_name[-8:]  == '.longFmt' or col_name[-4:] == '.fmt' or col_name[-2:] == '_x'):
            del df[col_name]
    return df

In [9]:
def scrape_list(list_of_stocks):
    temp = pd.DataFrame()
    for stock in list_of_stocks:
        single_stock_data = scrape(stock)
        temp = pd.concat([temp, single_stock_data])
        temp = remove_fluff(temp)
    df = move_columns(['ticker', 'endDate'], temp)
    return df


In [10]:
def move_columns(list_of_columns, df):
    for column in list_of_columns:
        just_a_column = df[column]
        df = df.drop(columns=[column])
        df.insert(loc=0, column=column, value=just_a_column)
    return df


In [11]:
#Initialize the driver
# options = EdgeOptions()
# options.use_chromium = True
# driver = Edge(options=options)

# url = 'https://finance.yahoo.com/'

# driver.get(url)


In [12]:
#Initialize the driver BUT THIS TIME in chrome
driver = webdriver.Chrome()

url = 'https://finance.yahoo.com/'

driver.get(url)

In [13]:
stocks = pd.read_csv('Tickers_remaining4.csv')

stocks = stocks['Stocks'].tolist()

download_list(stocks)



1.52% complete...
3.03% complete...
4.55% complete...
6.06% complete...
7.58% complete...
9.09% complete...
10.61% complete...
12.12% complete...
13.64% complete...
15.15% complete...
16.67% complete...
18.18% complete...
19.7% complete...
21.21% complete...
22.73% complete...
24.24% complete...
25.76% complete...
27.27% complete...
28.79% complete...
30.3% complete...
31.82% complete...
33.33% complete...
34.85% complete...
36.36% complete...
37.88% complete...
39.39% complete...
40.91% complete...
42.42% complete...
43.94% complete...
45.45% complete...
46.97% complete...
48.48% complete...
50.0% complete...
51.52% complete...
53.03% complete...
54.55% complete...
56.06% complete...
57.58% complete...
59.09% complete...
60.61% complete...
62.12% complete...
63.64% complete...
65.15% complete...
66.67% complete...
68.18% complete...
69.7% complete...
71.21% complete...
72.73% complete...
74.24% complete...
75.76% complete...
77.27% complete...
78.79% complete...
80.3% complete...
81.8